In [40]:
#Jacob Simmons, 11Jun2023, C340, Assignment 6-1

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64


# Configure the plotting routines
import dash
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from pymongo import MongoClient
import Animal_Shelter
from Animal_Shelter import AnimalShelter
from bson.objectid import ObjectId
from pprint import pprint



###########################
# Data Manipulation / Model
###########################

username = "aacUser"
password = "SNHU1234"
HOST = 'nv-desktop-services.apporto.com'
PORT = 31379
DB = 'AAC'
COL = 'animals'
shelter = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)

#setting the parameters to input the logo
logo_file = 'assets/Logo.png'
encoded_logo = base64.b64encode(open(logo_file, 'rb').read())


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#main header/title and logo display
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS340 Dashboard', style={'color': 'black'}))),
    html.Center(html.B(html.H2('Jacob Simmons, Project Two', style={'color': 'blue'}))),
    html.Center(html.Img(style={'height':'23%', 'width':'23%'}, src="data:image/png;base64,{}".format(encoded_logo.decode()))),
    
    #formatting the radio items
    html.Hr(),
        html.Div([
        html.H4(html.Center(html.Label(['Select A Filter Option: '], style={'font-weight': 'italics'}))),
        html.H4(html.Center(dcc.RadioItems(
            id='radio-items',
            options=[
                {'label': 'Water Rescue    ', 'value': 1},
                {'label': 'Mountain or Wilderness Rescue    ', 'value': 2},
                {'label': 'Disaster or Individual Tracking    ', 'value': 3},
                {'label': 'Reset    ', 'value': 4}
            ],
            value=4,
            labelStyle={'display': 'block'}
        ))
    )]),
            
    html.Br(),
    
    #Creating data table to display 10 lines of imported data
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        
        data=df.to_dict('records'),
        
        #setting table filter options and interactive properties
        editable=False,
        filter_action="native",
        sort_action='custom',
        sort_mode="multi",
        sort_by=[],
        row_selectable="single",
        row_deletable=True,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=12,

    ),
    html.Br(),
    html.Hr(),
    # This div sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Center(html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     className='row',
                     style={'display': 'flex'},
                     children=[
                         
                         dcc.Graph(
                             id='graph-id',
                             className='col s12 m6',
                        style={
                        "width": "800px",
                        "height": "800px",
                        "display": "inline-block",
                        "padding-top": "5px",
                        "padding-left": "100px",
                            }
                         ),
                       
                         html.Div(
                             id='map-id',
                             className='col s12 m6',
                        style={
                        "width": "800px",
                        "height": "800px",
                        "display": "inline-block",
                        "padding-top": "5px",
                        "padding-left": "100px",
                            }
                         ),
                     ]
                 ),
             ])
        )
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it

# Radio button filters
@app.callback(Output('datatable-id','data'),
              [Input('radio-items', 'value')])

def update_table(radio_selection):
    
        #Filter for water rescue
        if radio_selection == 1:
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"
                                 ]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
        #Filter for Mountain or Wilderness Rescue
        elif radio_selection == 2:
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
        #Filter for Disaster Rescue or Individual Tracking
        elif radio_selection == 3:
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", 
                                  "Bloodhound","Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            }))
            
        #No filter option
        else:
            df = pd.DataFrame.from_records(shelter.read({}))
        
        #remove last column for processing
        df.drop(columns=['_id'],inplace=True)
        
        return df.to_dict('records')

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable

#This is the graphing callback and update function
@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-id', "derived_virtual_data")])


def update_graph(allData):
    
    #Data formatting for scatter plot
    dff = pd.DataFrame.from_dict(allData)
    
    #Scatter plot input parameters
    fig = px.scatter(data_frame=dff, x=dff['breed'], y=dff['outcome_type'])
    
    return fig
       
    

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, derived_virtual_selected_rows):

    dff = df if viewData is None else pd.DataFrame(viewData)
    filtered_animal = None

    #Check if no selection
    if not derived_virtual_selected_rows:
        filtered_animal = dff.iloc[0]
        
    #Use selection display for option
    else:
        filtered_animal = dff.iloc[derived_virtual_selected_rows[0]]

    #setting values based on the .csv input
    latitude = filtered_animal[13]
    longitude = filtered_animal[14]
    animal_breed = filtered_animal[4]
    animal_name = filtered_animal[9]

    return [
        dl.Map(center=[latitude, longitude], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            
            dl.Marker(position=[latitude, longitude], children=[
                
                dl.Tooltip(animal_breed),
                
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(animal_name)
                ])
            ])
        ])
    ]



if __name__ == '__main__':
    app.run_server(debug=True)

ModuleNotFoundError: No module named 'dash_daq'